<a href="https://colab.research.google.com/github/VindiSovia/AutoTotalCard/blob/main/DEPLOY_ELAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clone github dengan Model yang telah dimodifikasi**

In [ ]:
!git clone https://github.com/VindiSovia/yolov5.git
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12775, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 12775 (delta 22), reused 36 (delta 9), pack-reused 12714
Receiving objects: 100% (12775/12775), 14.90 MiB | 18.52 MiB/s, done.
Resolving deltas: 100% (8491/8491), done.
/content/yolov5/yolov5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Download Model**
`Epoch 500`

In [ ]:
!gdown https://drive.google.com/uc?id=10WH9c_xJptinfr9VY8QwlcQHJTCaleEb

Downloading...
From: https://drive.google.com/uc?id=10WH9c_xJptinfr9VY8QwlcQHJTCaleEb
To: /content/yolov5/yolov5/best.pt
100% 139M/139M [00:01<00:00, 79.4MB/s]


**REQUIREMENTS yang digunakan**

In [ ]:
%%writefile requirements.txt
# YOLOv5 requirements
# Usage: pip install -r requirements.txt

# Base ----------------------------------------
matplotlib>=3.2.2
numpy>=1.18.5
opencv-python
Pillow>=7.1.2
PyYAML>=5.3.1
requests>=2.23.0
scipy>=1.4.1  # Google Colab version
torch>=1.7.0
torchvision>=0.8.1
tqdm>=4.41.0
protobuf<4.21.5  # https://github.com/ultralytics/yolov5/issues/8012
ultralytics
# Logging -------------------------------------
tensorboard>=2.4.1
# wandb

# Plotting ------------------------------------
pandas>=1.1.4
seaborn>=0.11.0


# Extras --------------------------------------
ipython  # interactive notebook
psutil  # system utilization
thop  # FLOPs computation
streamlit
wget
ffmpeg-python
pyngrok
streamlit_webrtc
qrcode[pil]
python-barcode

Overwriting requirements.txt


In [ ]:
!pip install -r requirements.txt

In [ ]:
%%writefile video_predict.py
import os
import os.path as osp
import cv2
from PIL import Image
import streamlit as st

class_labels_and_costs = {
    "Anggur": 30000,
    "Apel": 10000,
    "Jambu": 7000,
    "Jeruk": 6000,
    "Kelengkeng": 45000,
    "Mangga": 6000,
    "Manggis": 3000,
    "Nanas": 16000,
    "Pisang": 34000,
    "Salak": 8000,
    "Strawberry": 20000,
    # Add more class labels and costs as needed
}

def runVideo(model, video, vdo_view, warn):
    video_name = osp.basename(video)
    outputpath = osp.join('data/video_output', video_name)

    # Create A Dir to save Video Frames
    os.makedirs('data/video_frames', exist_ok=True)
    frames_dir = osp.join('data/video_frames', video_name)
    os.makedirs(frames_dir, exist_ok=True)
    cap = cv2.VideoCapture(video)
    frame_count = 0

    # Count the number of objects and calculate total cost
    num_objects = 0
    total_cost = 0

    with st.spinner(text="Predicting..."):
        warn.warning(
            'This is realtime prediction, If you wish to download the final prediction result wait until the process done.', icon="⚠️")
        while True:
            frame_count += 1
            ret, frame = cap.read()
            if ret == False:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = model(frame)
            result.render()
            image = Image.fromarray(result.ims[0])
            vdo_view.image(image, caption='Current Model Prediction(s)')

            # Calculate cost for each detected object
            for i, result in enumerate(result.xyxy[0]):
                label = model.names[int(result[-1])]
                if label in class_labels_and_costs:
                    num_objects += 1
                    cost = class_labels_and_costs[label]
                    total_cost += cost

                    # Save object information in session state
                    st.session_state[f'object_label_{num_objects}'] = label
                    st.session_state[f'object_cost_{num_objects}'] = cost

                    # Optionally, you can display the cost for each detected object
                    st.success(f"Object {num_objects}: {label} - Harga: Rp. {cost}")

        cap.release()
        # convert frames in dir to a single video file
        os.system(
            f'ffmpeg -framerate 30 -i {frames_dir}/%d.jpg -c:v libx264 -pix_fmt yuv420p {outputpath}')
    # Clean up Frames Dir
    os.system(f'rm -rf {frames_dir}')

    # Display Video
    output_video = open(outputpath, 'rb')
    output_video_bytes = output_video.read()
    st.video(output_video_bytes)
    st.write("Model Prediction")
    vdo_view.empty()
    warn.empty()

    # Display the count and total cost
    st.success(f"Number of Objects Detected: {num_objects}")
    st.success(f"Total Cost: ${total_cost}")

Writing video_predict.py


**Deploy APP**

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from PIL import Image
from io import *
import glob
from datetime import datetime
import os
import wget
from video_predict import runVideo
import numpy as np
from streamlit_webrtc import webrtc_streamer, WebRtcMode, RTCConfiguration
import av
import qrcode
from PIL import Image
import barcode
from barcode.writer import ImageWriter


# Configurations
CFG_MODEL_PATH = "best.pt"
CFG_ENABLE_URL_DOWNLOAD = True
CFG_ENABLE_VIDEO_PREDICTION = True
if CFG_ENABLE_URL_DOWNLOAD:
    # Configure this if you set cfg_enable_url_download to True
    url = "https://drive.google.com/uc?id=10WH9c_xJptinfr9VY8QwlcQHJTCaleEb"+".pt"
# End of Configurations

class_labels_and_costs = {
    "Anggur": 30000,
    "Apel": 10000,
    "Jambu": 7000,
    "Jeruk": 6000,
    "Kelengkeng": 45000,
    "Mangga": 6000,
    "Manggis": 3000,
    "Nanas": 16000,
    "Pisang": 34000,
    "Salak": 8000,
    "Strawberry": 20000,
    # Add more class labels and costs as needed
}

# Initialize session state variables
for i in range(1, 4):  # Assuming you have 3 objects
    object_label_key = f'object_label_{i}'
    object_cost_key = f'object_cost_{i}'

    if object_label_key not in st.session_state:
        st.session_state[object_label_key] = f"Label {i}"

    if object_cost_key not in st.session_state:
        st.session_state[object_cost_key] = 0  # You can set an appropriate default value

def generate_receipt(num_objects, total_cost):

    receipt = f"\nEternalLumenAI\n{'='*30}\n"
    receipt += f"Number of Objects Detected: {num_objects}\n"
    receipt += f"{'='*30}\n"
    for i in range(1, num_objects + 1):
        receipt += f"Object {i}: {st.session_state[f'object_label_{i}']} - Harga: Rp. {st.session_state[f'object_cost_{i}']}\n"
    receipt += f"{'='*30}\n"
    receipt += f"Total Harga: Rp. {total_cost}\n"
    receipt += f"{'='*30}\n"

    # Generate QR Code for the receipt
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(receipt)
    qr.make(fit=True)

    # Create an image from the QR Code instance
    qr_img = qr.make_image(fill_color="black", back_color="white")

    # Save the QR Code image
    qr_img_path = "receipt_qr_code.png"
    qr_img.save(qr_img_path)

    # Display the receipt text
    st.text(receipt)

    # Display the QR Code image
    st.image(qr_img_path, caption="Scan the QR Code to view your receipt", width=300)

    return receipt

def imageInput(model, src):
    if src == 'Upload your own data.':
        image_file = st.file_uploader(
            "Upload An Image", type=['png', 'jpeg', 'jpg'])

        if image_file is not None:
            try:
                img = Image.open(image_file)
            except UnidentifiedImageError:
                st.error("Cannot identify the uploaded file as an image. Please make sure the file is a valid image.")
                return

            col1, col2 = st.columns(2)
            with col1:
                st.image(img, caption='Uploaded Image',
                         use_column_width='always')
            ts = datetime.timestamp(datetime.now())
            imgpath = os.path.join('data/uploads', str(ts)+image_file.name)
            outputpath = os.path.join(
                'data/outputs', os.path.basename(imgpath))
            with open(imgpath, mode="wb") as f:
                f.write(image_file.getbuffer())

            with st.spinner(text="Predicting..."):
                # Load model
                pred = model(imgpath)
                pred.render()

                # Count the number of objects and calculate total cost
                num_objects = 0
                total_cost = 0

                for i, result in enumerate(pred.xyxy[0]):
                    label = model.names[int(result[-1])]
                    if label in class_labels_and_costs:
                        num_objects += 1
                        cost = class_labels_and_costs[label]
                        total_cost += cost

                        # Save object information in session state
                        st.session_state[f'object_label_{num_objects}'] = label
                        st.session_state[f'object_cost_{num_objects}'] = cost

                        # Optionally, you can display the cost for each detected object
                        st.success(f"Object {num_objects}: {label} - Harga: Rp. {cost}")

                # Display the count and total cost
                st.success(f"Number of Objects Detected: {num_objects}")
                st.success(f"Total Harga: Rp. {total_cost}")

                # Save output to file
                for im in pred.ims:
                    im_base64 = Image.fromarray(im)
                    im_base64.save(outputpath)

                # Predictions
                img_ = Image.open(outputpath)
                with col2:
                    st.image(img_, caption='Model Prediction(s)',
                             use_column_width='always')

                # Display the receipt
                receipt = generate_receipt(num_objects, total_cost)

    elif src == 'From example data.':
        # Image selector slider
        imgpaths = glob.glob('data/example_images/*')
        if len(imgpaths) == 0:
            st.write(".")
            st.error(
                'No images found, Please upload example images in data/example_images', icon="")
            return
        imgsel = st.slider('Select random images from example data.',
                           min_value=1, max_value=len(imgpaths), step=1)
        image_file = imgpaths[imgsel-1]
        submit = st.button("Predict!")
        col1, col2 = st.columns(2)
        with col1:
            img = Image.open(image_file)
            st.image(img, caption='Selected Image', use_column_width='always')
        with col2:
            if image_file is not None and submit:
                with st.spinner(text="Predicting..."):
                    # Load model
                    pred = model(image_file)
                    pred.render()

                    # Save output to file
                    for im in pred.ims:
                        im_base64 = Image.fromarray(im)
                        im_base64.save(os.path.join(
                            'data/outputs', f'{os.path.basename(image_file)}'))

                    # Display prediction
                    img_ = Image.open(os.path.join(
                        'data/outputs', f'{os.path.basename(image_file)}'))
                    st.image(img_, caption='Model Prediction(s)')

                    # Count the number of objects and calculate total cost
                    num_objects = 0
                    total_cost = 0

                    for i, result in enumerate(pred.xyxy[0]):
                        label = model.names[int(result[-1])]
                        if label in class_labels_and_costs:
                            num_objects += 1
                            cost = class_labels_and_costs[label]
                            total_cost += cost

                            # Save object information in session state
                            st.session_state[f'object_label_{num_objects}'] = label
                            st.session_state[f'object_cost_{num_objects}'] = cost

                            # Optionally, you can display the cost for each detected object
                            st.success(f"Object {num_objects}: {label} - Harga: Rp. {cost}")

                    # Display the count and total cost
                    st.success(f"Number of Objects Detected: {num_objects}")
                    st.success(f"Total Harga: Rp. {total_cost}")


def videoInput(model, src):
    if src == 'Upload your own data.':
        uploaded_video = st.file_uploader(
            "Upload A Video", type=['mp4', 'mpeg', 'mov'])
        pred_view = st.empty()
        warning = st.empty()
        if uploaded_video != None:

            # Save video to disk
            ts = datetime.timestamp(datetime.now())  # timestamp a upload
            uploaded_video_path = os.path.join(
                'data/uploads', str(ts)+uploaded_video.name)
            with open(uploaded_video_path, mode='wb') as f:
                f.write(uploaded_video.read())

            # Display uploaded video
            with open(uploaded_video_path, 'rb') as f:
                video_bytes = f.read()
            st.video(video_bytes)
            st.write("Uploaded Video")
            submit = st.button("Run Prediction")
            if submit:
                runVideo(model, uploaded_video_path, pred_view, warning)

    elif src == 'From example data.':
        # Image selector slider
        videopaths = glob.glob('data/example_videos/*')
        if len(videopaths) == 0:
            st.error(
                'No videos found, Please upload example videos in data/example_videos', icon="⚠️")
            return
        imgsel = st.slider('Select random video from example data.',
                           min_value=1, max_value=len(videopaths), step=1)
        pred_view = st.empty()
        video = videopaths[imgsel-1]
        submit = st.button("Predict!")
        if submit:
            runVideo(model, video, pred_view, warning)


def webcam_streamlit():
    device = "cpu"
    st.model = torch.hub.load('VindiSovia/yolov5', 'custom',
                              path=CFG_MODEL_PATH, force_reload=True, device=device, skip_validation=True)
    RTC_CONFIGURATION = RTCConfiguration({"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]})


    class VideoProcessor:
        def recv(self, frame):
            img = frame.to_ndarray(format="rgb24")
            flipped = img[:, ::-1, :]
            im_pil = Image.fromarray(flipped)
            results = st.model(im_pil, size=112)
            bbox_img = np.array(results.render()[0])

            # Count the number of objects and calculate total cost
            num_objects = 0
            total_cost = 0

            for i, result in enumerate(results.xyxy[0]):
                label = st.model.names[int(result[-1])]
                if label in class_labels_and_costs:
                    num_objects += 1
                    cost = class_labels_and_costs[label]
                    total_cost += cost

                    # Save object information in session state
                    st.session_state[f'object_label_{num_objects}'] = label
                    st.session_state[f'object_cost_{num_objects}'] = cost

                    # Optionally, you can display the cost for each detected object
                    st.success(f"Object {num_objects}: {label} - Harga: Rp. {cost}")

            # Display the count and total cost outside the for loop
            st.success(f"Number of Objects Detected: {num_objects}")
            st.success(f"Total Harga: Rp. {total_cost}")

            return av.VideoFrame.from_ndarray(bbox_img, format="rgb24")


    webrtc_ctx = webrtc_streamer(
        key="WYH",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        video_processor_factory=VideoProcessor,
        media_stream_constraints={"video": True, "audio": False},
        async_processing=False,
    )


def main():
    if CFG_ENABLE_URL_DOWNLOAD:
        downloadModel()
    else:
        if not os.path.exists(CFG_MODEL_PATH):
            st.error(
                'Model not found, please configure if you wish to download the model from the URL by setting `cfg_enable_url_download = True`', icon="⚠️")

    # -- Sidebar
    st.sidebar.image("https://i.postimg.cc/mrjW33vT/keranjang-buah.png", width=100)
    st.sidebar.header('''Welcome to Our App 🖐️''')
    st.sidebar.title('⚙️ Options')
    datasrc = st.sidebar.radio("Select input source.", [
                               'From example data.', 'Upload your own data.'])

    if CFG_ENABLE_VIDEO_PREDICTION:
        option = st.sidebar.radio("Select input type.", ['Image', 'Video', 'Webcam', 'Price List', 'Our Team'])
    else:
        option = st.sidebar.radio("Select input type.", ['Image', 'Price List', 'Our Team'])
    if torch.cuda.is_available():
        deviceoption = st.sidebar.radio("Select compute Device.", [
                                        'cpu', 'cuda'], disabled=False, index=1)
    else:
        deviceoption = st.sidebar.radio("Select compute Device.", [
                                        'cpu', 'cuda'], disabled=True, index=0)


    st.sidebar.markdown("""
    Email : eternallumenai@gmail.com""")

    # -- End of Sidebar

    st.header('🛒 Auto Total Card - Keranjang Buah Penghitung Otomatis 🍇')
    st.sidebar.write(" ")
    st.sidebar.write(" ")

    if option == "Image":
        imageInput(loadmodel(deviceoption), datasrc)
    elif option == "Video":
        videoInput(loadmodel(deviceoption), datasrc)
    elif option == "Webcam":
        webcam_streamlit()
    elif option == "Price List":
        st.sidebar.write(" ")
        st.header("💰 Price List")
        st.subheader("Check the prices of fruits:")
        st.markdown("""
        - **Anggur**: Rp. 30000,
        - **Apel**: Rp. 10000,
        - **Jambu**: Rp. 7000,
        - **Jeruk**: Rp. 6000,
        - **Kelengkeng**: Rp. 45000,
        - **Mangga**: Rp. 6000,
        - **Manggis**: Rp. 3000,
        - **Nanas**: Rp. 16000,
        - **Pisang**: Rp. 34000,
        - **Salak**: Rp. 8000,
        - **Strawberry**: Rp. 20000,

        """)
    elif option == "Our Team":
      st.header("👥 Our Team")
      st.sidebar.write(" ")
      st.subheader("Check our team:")
      st.markdown("""
      - **Aditia Taqi Pratama (Leader)**  - AI05102 - Universitas Negeri Jakarta
      - **Ardis Hibatul Hakim**           - AI05104 - Universitas Gunadarma
      - **Kinanthi Putri Siwilopo**       - AI05210 - Universitas Amikom Purwokerto
      - **Maharani Ria Sina**             - AI05213 - Universitas Sebelas Maret
      - **Vindi Sovia Anggita**           - AI05322 - Politeknik Negeri Malang
      - **Ahmad Yudiman Fauzi**           - AI05401 - Universitas Garut

      """)



# Downlaod Model from url.
@st.cache_resource
def downloadModel():
    if not os.path.exists(CFG_MODEL_PATH):
        wget.download(url, out="models/")


@st.cache_resource
def loadmodel(device):
    model = torch.hub.load('VindiSovia/yolov5', 'custom',
                           path=CFG_MODEL_PATH, force_reload=True, device=device)
    return model


if __name__ == '__main__':
    main()

Writing app.py


In [ ]:
import requests

response = requests.get('https://ipv4.icanhazip.com')
public_ip = response.text.strip()
print("Public IP:", public_ip)

Public IP: 34.125.138.184


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] / rollbackFailedOptional: verb npm-session 576289501ad1f36


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.138.184:8501

npx: installed 22 in 2.185s
your url is: https://beige-crabs-behave.loca.lt
Downloading: "https://github.com/VindiSovia/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/VindiSovia/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
2023-12-03 14:14:41.809 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/caching/cache_utils.py", line 264, in _get_or_create_cached_value
    cached_result = cache.read_result(value_key)
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/caching/cache_resource_api.py", line 500, in read_result
    raise CacheKeyNotFoundError()
streamlit.runtime.caching.cache_errors.CacheKeyNotFoundError

During handli